# Multihop Task Execution with OpenAPI

Using robust OpenAPI Chain primitives, LLM Chains can efficiently plan and combine calls across multiple endpoints. This notebook demonstrates a sample composition of Speak and Klarna APIs.

For a detailed walkthrough, see the [OpenAPI Operation Chain](openapi.ipynb) notebook.

### First, import dependencies and load the LLM

In [1]:
from langchain.chains import LLMChain, OpenAPIEndpointChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.requests import Requests
from langchain.tools import APIOperation, OpenAPISpec

In [2]:
# Select the LLM to use. Here, we use text-davinci-003
llm = OpenAI(temperature=0, max_tokens=700) # You can swap between different core LLM's here.

### Next, load the OpenAPI specification

In [3]:
speak_spec = OpenAPISpec.from_url("https://api.speak.com/openapi.yaml")
klarna_spec = OpenAPISpec.from_url("https://www.klarna.com/us/shopping/public/openai/v0/api-docs/")

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


### Create the OpenAPIEndpointChain

Create an OpenAPIEndpointChain for each of the API's HTTP methods. For larger OpenAPI specs, you may find it easier to select a subset of the operations that work well together.

In [4]:
from typing import Optional

# Headers can be passed in for user authentication to the endpoint.
def make_chains(spec, headers: Optional[dict] = None) -> list:
    """Create a chain for each endpoint in the OpenAPI spec."""
    operation_chains = []
    for path in spec.paths:
        for method in spec.get_methods_for_path(path):
            api_operation = APIOperation.from_openapi_spec(spec, path, method)
            chain = OpenAPIEndpointChain.from_api_operation(
                api_operation, 
                llm, 
                requests=Requests(headers=headers), 
                verbose=False,
                return_intermediate_steps=True # Return request and response text
            )
            operation_chains.append(chain)
    return operation_chains
klarna_llm_chains = make_chains(klarna_spec)
speak_llm_chains = make_chains(speak_spec)
llm_chains = klarna_llm_chains + speak_llm_chains

### Create the Planner Chain

This chain generates a single plan based on the user's query that will be executed by the selected tools.

In [5]:
# Format the APIs into a simple list
apis_format = "\n".join([
    f" - {chain.api_operation.operation_id}: {chain.api_operation.description}" for chain in llm_chains
])

In [6]:
plan_template = """You are trying to answer the following question by querying one more more API's then communicating the answer:

> Question: {question}

You have access to the following helpers who can interact with APIs.

{apis}

Think how to best use the helpers, then select which helpers you want to ask for assistance and in what order. Use the following format:

Thoughts: <Think about why the plan (and its order) is the best.
Be detailed.
>

Plan:
1. HELPER_NAME: <Instructions relating what you want the helper to assist with, why your asking, and how to summarize the information in natural language.>
2. HELPER_NAME: ...
...

BEGIN:
(remember to think step by step)
Thoughts:"""

verbose = False
prompt = PromptTemplate.from_template(plan_template)
planner_chain = LLMChain(llm=llm, prompt=prompt, verbose=verbose)

### Fetch the user query and generate a plan

In [7]:
user_input = "I have an end of year party for my Italian class and have to buy some Italian clothes for it"

In [8]:
raw_plan = planner_chain.run(question=user_input, apis=apis_format)

### Parse the plan into constituent steps

In [9]:
import re

plan = raw_plan.split('Plan:')[1].strip()
parsed_plan = list(re.findall("^\d+\.\s*(.*)", plan, re.MULTILINE))
actions_and_inputs = [tuple(step.split(': ', 1)) for step in parsed_plan]
actions_and_inputs

[('productsUsingGET',
  'Find Italian clothing that I can buy for the end of year party.'),
 ('translate', 'Translate the phrase "end of year party" into Italian.'),
 ('explainPhrase',
  'Explain the meaning of the phrase "end of year party" in Italian.'),
 ('explainTask',
  'Explain the best way to say or do something in the specific situation of buying Italian clothes for the end of year party.')]

### Execute the steps

Since this chain creates stepwise dependencies, we execute sequentially. You can also prompt the planner to make parallel plans and use the MapReduceChain

In [10]:
# Facilitate easy lookup by operation ID
apis_dict = {
    chain.api_operation.operation_id: chain for chain in llm_chains
}

# The intermediate steps also contain nested information if you want to further
# inspect the delegated tasks.
intermediate_steps = []
for action, action_input in actions_and_inputs:
    if intermediate_steps:
        prev_output = intermediate_steps[-1]["output"]
        action_input = f"{action_input}\n\n Information from the previous step: {prev_output}"
    intermediate_steps.append(apis_dict[action]({"instructions": action_input}))

# Format the output of each step
step_outputs = "\n".join([
    f"- Helper {i} said: \"{step['output']}\"" 
    for i, step 
    in enumerate(intermediate_steps)
])
print(step_outputs)

- Helper 0 said: "I found several Italian clothing items that you can buy for the end of year party. Check out the following products: OH LÀ LÀ CHÉRI A Lace In Your Heart Bralette Set - Italian Plum/Black, OH LÀ LÀ CHÉRI Valentine Soft Cup Babydoll Chemise & G-String Thong - Italian Plum, Cole Haan Men's Wool-Blend Italian Topcoat, Hugo Boss Men's Slim-Fit Italian Virgin Wool Sport Coat, Billion aire Italian Couture Cotton Hooded Sweatsuit gray, eberjey Short Organic Cotton Pajama Set Italian Rose, Aubade Womens Danse Des Sens Italian Brief, OH LÀ LÀ CHÉRI Thea Lace Trim Satin Wrap - Italian Plum/Black, Alé Colour Block Short Sleeve Jersey Men - Italian Blue, and Boss Italian-Leather Belt with Branded Metal Trim - Brown."
- Helper 1 said: "ERROR parsing response."
- Helper 2 said: "I attempted to call an API, but there was an error parsing the response. Please try again later."
- Helper 3 said: "In Italy, fashion is a very important part of life, so you can definitely use this as an ex

### Synthesize a final response

Use the collected information to synthesize a response to the original user query.

In [11]:
synthesizer_template = """Please answer the user's question as best as you are able using the gathered information.

Fetched information
---

{step_outputs}

Original Question
----
> User: {question} - Share relevant information where appropriate! I haven't read anything above.

RESPONSE
----

"""

prompt = PromptTemplate.from_template(synthesizer_template)
response_synthesizer = LLMChain(llm=llm, prompt=prompt, verbose=verbose)

In [12]:
# Here is the result a user would see for their original query
response_synthesizer.run(question=user_input, step_outputs=step_outputs)

"It sounds like you're looking for some Italian clothing items for your end of year party! Helper 0 has provided a list of Italian clothing items that you can buy, such as OH LÀ LÀ CHÉRI A Lace In Your Heart Bralette Set - Italian Plum/Black, Cole Haan Men's Wool-Blend Italian Topcoat, and Hugo Boss Men's Slim-Fit Italian Virgin Wool Sport Coat. Additionally, Helper 3 has provided some useful phrases in Italian that you can use when shopping for Italian clothes, such as 'Vorrei comprare dei vestiti italiani per la nostra festa di fine anno.' for a polite and formal tone, or 'Mi servono degli abiti alla moda italiana da sfoggiare alla festa.' for a stylish and casual tone."

### Adding more endpoints

You may think "these sound a lot like Agent tools." And you'd be right! Let's add more endpoints and mix with other tools to form a more complex chain.

In [13]:
from langchain.agents.tools import Tool

Adding the Spoonacular endpoints.

1. Go to the [Spoonacular API Console](https://spoonacular.com/food-api/console#Dashboard) and make a free account.
2. Click on `Profile` and copy your API key below.

In [14]:
spoonacular_spec = OpenAPISpec.from_url("https://spoonacular.com/application/frontend/downloads/spoonacular-openapi-3.json")
spoonacular_api_key = "" # Copy from the API Console
spoonacular_llm_chains = make_chains(
    spoonacular_spec,
    headers={"x-api-key": spoonacular_api_key}
)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Accept. Valid values are ['path', 'query'] Ignoring optional parameter
Unsupported APIPropertyLocation "header" for parameter Content-Type. Valid values 

### Wrap Endpoints as Tools

In [15]:
def chain_to_tool(chain: OpenAPIEndpointChain, domain_title: str) -> Tool:
    expanded_name = f'{domain_title.replace(" ", "_")}.{chain.api_operation.operation_id}'
    return Tool(
        name=expanded_name,
        func=chain,
        description=chain.api_operation.description
    )

In [16]:
klarna_tools = [chain_to_tool(chain, klarna_spec.info.title) for chain in klarna_llm_chains]
speak_tools = [chain_to_tool(chain, speak_spec.info.title) for chain in speak_llm_chains]
spoonacular_tools = [chain_to_tool(chain, spoonacular_spec.info.title) for chain in spoonacular_llm_chains]
all_tools = klarna_tools + speak_tools + spoonacular_tools
print(f"Loaded {len(all_tools)} endpoint tools")

Loaded 103 endpoint tools


In [27]:
# Make the query more complex!
user_input = (
    "I'm learing Italian, and my language class is having an end of year party... "
    " Could you help me find an Italian outfit to wear and"
    " an appropriate recipe to prepare so I can present for the class in Italian?"
)

### Generate a plan

Use the planner_chain to decide which tools to use.

In [28]:
tools_list = "\n".join([f" - {tool.name}: {tool.description}" for tool in all_tools])

# We'll re-use the user question from above. Let's see what the Planning chain cooks up!
raw_plan = planner_chain.run(question=user_input, apis=tools_list)

# Re-use the simple parsing from above
plan = raw_plan.split('Plan:')[1].strip()
parsed_plan = list(re.findall("^\d+\.\s*(.*)", plan, re.MULTILINE))
actions_and_inputs = [tuple(step.split(': ', 1)) for step in parsed_plan]
actions_and_inputs

[('Open_AI_Klarna_product_Api.productsUsingGET',
  'Search for Italian-style clothing.'),
 ('spoonacular_API.searchRecipes', 'Search for Italian recipes.'),
 ('spoonacular_API.getRecipeInformation',
  'Get detailed information about the recipe, such as ingredients, nutrition, diet and allergen information.'),
 ('Speak.translate', 'Translate the recipe instructions into Italian.')]

### Run the selected tools

In [29]:
tools_dict = {tool.name: tool for tool in all_tools}

# The intermediate steps also contain nested information if you want to further
# inspect the delegated tasks.
intermediate_steps = []
for action, tool_input in actions_and_inputs:
    if intermediate_steps:
        prev_output = intermediate_steps[-1]["output"]
        # In non-sequential plan, we would parse what information to pass to the selected tool
        tool_input = f"{action_input}\n\n Information from the previous step: {prev_output}"
    intermediate_steps.append(tools_dict[action](tool_input))

# Format the output of each step
step_outputs = "\n".join([
    f"- {step['output']}" 
    for step 
    in intermediate_steps
])

### Synthesize the output



In [30]:
response_synthesizer.run(question=user_input, step_outputs=step_outputs)

"Ciao! Per trovare un outfit alla moda italiana per la tua festa di fine anno, ti consigliamo di dare un'occhiata ai prodotti che abbiamo trovato nella nostra API: OH LÀ LÀ CHÉRI A Lace In Your Heart Bralette Set - Italian Plum/Black, Design Toscano Portare Acqua Italian-Style Stone Bonded Sculptural Fountain, e Levi's 501 Original Style Shrink-to-fit Jeans. Per trovare una ricetta appropriata, potresti provare a cercare frasi come: 'Voglio preparare una ricetta italiana per la festa di fine anno', 'Quali sono le ricette più popolari in Italia?' o 'Ho bisogno di una ricetta italiana per questa occasione importante.' Speriamo che questo ti aiuti!"

In [31]:
tools_dict['Speak.translate'].run("Tell the LangChain audience to 'enjoy the meal' in Italian, please!")['output']

'Goditi il pasto!'